In [13]:
import numpy as np
import requests
import os
from PIL import Image
from utils import prepare_images

MODEL_DIR = './tfserving1.6'
if not os.path.exists(MODEL_DIR):
    os.mkdir(MODEL_DIR)


# Vazios

In [14]:
# melhor modelo do notebook 01b3
# import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
SIZE = (224, 224)

model = Sequential([
  Conv2D(16, (3, 3),
                         padding='same',
                         activation='relu',
                         input_shape=(*SIZE, 3)),
  MaxPooling2D(pool_size=(2, 2)),
  Conv2D(32, (3, 3), padding='same', activation='relu'),
  MaxPooling2D(pool_size=(2, 2)),
  Dropout(0.2),
  Conv2D(64, (3, 3), padding='same', activation='relu'),
  MaxPooling2D(pool_size=(2, 2)),
  Dropout(0.25),
  Conv2D(128, (3, 3), padding='same', activation='relu'),
  MaxPooling2D(pool_size=(2, 2)),
  Dropout(0.25),
  Conv2D(128, (3, 3), activation='relu'),
  MaxPooling2D(pool_size=(2, 2)),
  Dropout(0.25),
  Conv2D(256, (3, 3), activation='relu'),
  Flatten(),
  Dense(256, activation='relu'),
  Dropout(0.4),
  Dense(1, activation='sigmoid')
 
])

In [15]:
model.load_weights(os.path.join('..', 'models', 'B3modelweights.17-0.08.hdf5'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_23 (Conv2D)           (None, 224, 224, 16)      448       
_________________________________________________________________
max_pooling2d_21 (MaxPooling (None, 112, 112, 16)      0         
_________________________________________________________________
conv2d_24 (Conv2D)           (None, 112, 112, 32)      4640      
_________________________________________________________________
max_pooling2d_22 (MaxPooling (None, 56, 56, 32)        0         
_________________________________________________________________
dropout_21 (Dropout)         (None, 56, 56, 32)        0         
_________________________________________________________________
conv2d_25 (Conv2D)           (None, 56, 56, 64)        18496     
_________________________________________________________________
max_pooling2d_23 (MaxPooling (None, 28, 28, 64)        0         
__________

In [20]:
from keras.models import save_model

version = 1
export_path = os.path.join(MODEL_DIR, 'vazio')
if not os.path.exists(export_path):
    os.mkdir(export_path)
export_path = os.path.join(export_path, str(version))
if not os.path.exists(export_path):
    os.mkdir(export_path)

print('export_path = {}\n'.format(export_path))

save_model(
    model,
    export_path,
    overwrite=True,
    include_optimizer=False,
)

print('\nSaved model:')
!ls -l {export_path}

export_path = ./tfserving1.6/vazio/1



OSError: Unable to create file (unable to open file: name = './tfserving1.6/vazio/1', errno = 21, error message = 'Is a directory', flags = 13, o_flags = 242)

In [2]:
#Testar Servidor
IMAGE_PATH = '../bases/vazios_cropped/VAZIO'
print(os.listdir(IMAGE_PATH)[:16])
images = prepare_images(os.listdir(IMAGE_PATH)[:16], IMAGE_PATH, (224, 224))

json_batch = {"signature_name": "serving_default", "instances": images}
r = requests.post('http://localhost:8501/v1/models/vazio:predict', json=json_batch)
print(r.json())

['5e5febda86556b3036c756cb.jpg', '5e692563635812f4b029d6b3.jpg', '5e6108f9dff3829839aa7e60.jpg', '5e7cc89d4bd4ad51c2d8bcaf.jpg', '5e5e4eeb86556b3036c71a1d.jpg', '5e6ba6ba4bd4ad51c2d5c7eb.jpg', '5e665e8f635812f4b028b2df.jpg', '5e610841f90d247d887a867c.jpg', '5e62311f9ccf00f7abbf68d4.jpg', '5e6925ba635812f4b029e399.jpg', '5e6ba6064bd4ad51c2d5b209.jpg', '5e6ba7744bd4ad51c2d5d59c.jpg', '5e6ba5fe4bd4ad51c2d5afe3.jpg', '5e6ba7744bd4ad51c2d5d57c.jpg', '5e6ba6054bd4ad51c2d5b1c9.jpg', '5e7cc8564bd4ad51c2d8b49c.jpg']
{'predictions': [[0.629091799], [0.965853333], [0.966044962], [0.966284215], [0.974903405], [0.976563573], [0.898215055], [0.96291852], [0.967284203], [0.961856], [0.000231626342], [0.977061331], [0.936749756], [0.972840607], [6.49218919e-06], [0.964953482]]}


In [ ]:
# To serve multiple models, create a file like:
#tfserving/model_config.config

model_config_list: {
  ...
  config: {
    name: "vazio",
    base_path: "/home/ivan/pybr/projeto/Publicacao_modelos/tfserving/vazio",
    model_platform: "tensorflow" 
  }
}

# Pesos

In [20]:
from tensorflow.keras import layers
from tensorflow.keras import models

regression_model = models.Sequential([
  layers.Conv2D(16, (7, 7), activation='relu',
                padding='same',
                strides=(2, 2),
                input_shape=(144, 288, 3)),
  layers.MaxPooling2D(pool_size=(2, 4)),
  layers.Dropout(0.25),
  layers.Conv2D(16, (5, 5), 
                strides=(2, 2),
                activation='relu', padding='same'),
  layers.Dropout(0.25),
  layers.Conv2D(16, (5, 5), 
                strides=(2, 2),
                activation='relu'),
  layers.Dropout(0.25),
  layers.AvgPool2D(),
  layers.Flatten(),
  layers.Dense(512, activation='relu'),
  layers.Dense(256, activation='relu'),
  layers.Dropout(0.4),
  layers.Dense(1)
 
])
regression_model.load_weights('../extracoes/notebooks/gerabasespesagens_cropped2_basefiltrada_scaled_warmup_tunned.hdf5')

In [21]:
import tensorflow as tf


MODEL_DIR = './tfserving'
version = 1
export_path = os.path.join(MODEL_DIR, 'peso', str(version))
print('export_path = {}\n'.format(export_path))

tf.keras.models.save_model(
    regression_model,
    export_path,
    overwrite=True,
    include_optimizer=False,
    save_format=None,
    signatures=None,
    options=None
)

print('\nSaved model:')
!ls -l {export_path}

export_path = ./tfserving/peso/1


Saved model:
total 192
drwxr-xr-x 2 ivan ivan   4096 Abr  6 13:51 assets
-rw-rw-r-- 1 ivan ivan 186637 Abr  6 13:51 saved_model.pb
drwxr-xr-x 2 ivan ivan   4096 Abr  6 13:51 variables


In [5]:
#Testar Servidor
IMAGE_PATH = '../bases/pesagensbalanca/pesagensbalanca_cropped/'
images = prepare_images(os.listdir(IMAGE_PATH)[:16], IMAGE_PATH, (288, 144))

json_batch = {"signature_name": "serving_default", "instances": images}
r = requests.post('http://localhost:8501/v1/models/peso:predict', json=json_batch)
print(r.json())

{'predictions': [[-0.0945815518], [1.31291544], [-0.892532349], [0.566698611], [0.791771054], [-0.356804579], [-1.11517954], [0.648615181], [-1.4110328], [-0.415910274], [-1.18397748], [0.289904267], [1.09787583], [0.625535965], [0.0675639287], [0.5964185]]}


In [4]:
#Configurar Servidor
model_config_list: {
  config: {
    name: "peso" ,
    base_path: "/home/ivan/pybr/projeto/Publicacao_modelos/tfserving/peso" ,
    model_platform: " tensorflow" 
  },
    ...
}
# Rodar diretamente 
tensorflow_model_server --rest_api_port=8501 \
--model_config_file="/home/ivan/pybr/projeto/Publicacao_modelos/tfserving/model_config.config"

SyntaxError: invalid syntax (<ipython-input-4-34fbe87fb50f>, line 2)

In [ ]:
#Configurar Servidor Docker
model_config_list: {
  config: {
    name: "peso" ,
    base_path: "/models/Publicacao_modelos/tfserving/peso" ,
    model_platform: " tensorflow" 
  },
    ...
}

# Rodar via docker
sudo docker run -t --rm -p 8501:8501 \
    --mount type=bind,source=/home/ivan/pybr/projeto/,target=/models \
    tensorflow/serving --model_config_file=models/Publicacao_modelos/tfserving/model_config_docker.config
    
    